In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.3.0'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-18 17:23:22--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.49MB/s    in 0.2s    

2022-08-18 17:23:22 (5.49 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProjectAnalysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

from pyspark import SparkFiles
# Load file
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_UK_v1_00.tsv.gz"

In [ ]:
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_multilingual_UK_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         UK|      10349|R2YVNBBMXD8KVJ|B00MWK7BWG|     307651059|My Favourite Fade...|               Music|          5|            0|          0|   N|                Y|          Five Stars|The best album ever!| 2014-12-29|
|         UK|      10629|R2K4BOL8MN1TTY|B006CHML4I|     835010224|Seiko 5 Men's Aut...|             Watc

In [ ]:
# Count the number of records (rows) in the dataset
rows = df.count()
print(f"dataframe has {rows} rows")

dataframe has 1707494 rows


In [ ]:
# Transform the dataset to fit the tables in the schema file. Be sure that the DataFrames match in data type and in column name.

#check for null values - we don't want any in review_id, customer_id, or product_id.
from pyspark.sql.functions import col,sum
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|               0|          0|            0|          0|   0|                0|              1|         50|         59|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [ ]:
# Get column names and types
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [ ]:
# CREATE TABLE review_id_table (
#   review_id TEXT PRIMARY KEY NOT NULL,
#   customer_id INTEGER,
#   product_id TEXT,
#   product_parent INTEGER,
#   review_date DATE -- this should be in the formate yyyy-mm-dd
# );

from pyspark.sql.functions import col
from pyspark.sql.functions import to_date

review_id_table = df[['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']]
review_id_table = review_id_table.withColumn("review_date",to_date(col("review_date"),"yyyy-mm-dd"))
review_id_table.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2YVNBBMXD8KVJ|      10349|B00MWK7BWG|     307651059| 2014-01-29|
|R2K4BOL8MN1TTY|      10629|B006CHML4I|     835010224| 2013-01-24|
|R3P40IEALROVCH|      12136|B00IIFCJX0|     271687675| 2014-01-09|
|R25XL1WWYRDLA9|      12268|B000W7JWUA|     211383699| 2014-01-19|
| RVTVB9YDXSFYH|      12677|B005JTAP4S|     182965893| 2013-01-18|
| R1P16QCZR7RHM|      13070|B00004WMYB|     530484605| 2013-01-30|
|R3DHVC6SGQS5JU|      15356|B00CEF014E|     658330608| 2013-01-26|
|R2F2I7T03D42TE|      16019|1849839735|     143601567| 2013-01-23|
|R3G5WIW7NNA1CS|      16563|B004OY47JS|      74795975| 2013-01-28|
| R75U5MUIZ9T0D|      17139|B009O36EO0|     269758980| 2012-01-10|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# -- This table will contain only unique values
# CREATE TABLE products (
#   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#   product_title TEXT
# );

products = df[['product_id', 'product_title']]
products = products.dropDuplicates()
products.count()

58703

In [ ]:
products.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|144476490X|Bob: No Ordinary Cat|
|B00742STVE|  Brave [DVD] [2012]|
|B007BRJYF2|Alice in Wonderla...|
|0141976144|Wreck This Journa...|
|B00C2SF8A0|The Bling Ring [D...|
|B0063H2ZS4|   Warrior [Blu-ray]|
|0099545837|Barbarians At The...|
|0553811851|Tripwire: (Jack R...|
|B00FLDG52I|Crossroads Guitar...|
|B000HT1XVY|To Kill a Mocking...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# -- Customer table for first data set
# CREATE TABLE customers (
#   customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#   customer_count INT
# );

customers = df[['customer_id']]
customers = customers.groupBy("customer_id").count()
customers = customers.withColumnRenamed("count","customer_count")

customers.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      27484|             1|
|      29285|             1|
|      76756|             1|
|      94377|             1|
|      94695|             1|
|     455661|             1|
|     520983|             1|
|    1066144|             1|
|    1293329|             1|
|    1464706|             2|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# -- vine table
# CREATE TABLE vine_table (
#   review_id TEXT PRIMARY KEY,
#   star_rating INTEGER,
#   helpful_votes INTEGER,
#   total_votes INTEGER,
#   vine TEXT
# );



vine_table = df[['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']]
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2YVNBBMXD8KVJ|          5|            0|          0|   N|
|R2K4BOL8MN1TTY|          4|            0|          0|   N|
|R3P40IEALROVCH|          5|            0|          0|   N|
|R25XL1WWYRDLA9|          5|            0|          0|   N|
| RVTVB9YDXSFYH|          5|           12|         12|   N|
| R1P16QCZR7RHM|          1|            1|          7|   N|
|R3DHVC6SGQS5JU|          5|            0|          0|   N|
|R2F2I7T03D42TE|          4|            0|          0|   N|
|R3G5WIW7NNA1CS|          5|            0|          0|   N|
| R75U5MUIZ9T0D|          5|            3|          4|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
#Load the DataFrames that correspond to tables into an RDS instance. 
# Note: This process can take up to 10 minutes for each. Ensure that everything is correct before uploading.

 # Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<<endpoint>>:5432/amazon_reviews"
config = {"user":"root", 
          "password": "<<pass>>", 
          "driver":"org.postgresql.Driver"}

          ## REMEMBER TO DELETE PASSWORD and ENDPOINT AFORE UPLOADING
          ## AND THEN SHUT DOWN ALL AWS BUCKETS AND RDS

In [ ]:
# Write DataFrames to tables

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)